In [1]:
import requests
import bs4
from dotenv import load_dotenv
import os
import psycopg
from datetime import datetime as dt

load_dotenv()

True

In [43]:
dbconn = os.getenv("DBCONN")
conn = psycopg.connect(dbconn)
cur = conn.cursor()

In [3]:
cur.execute('''
    CREATE TABLE IF NOT EXISTS utoday_news(
        title VARCHAR(255),
        author VARCHAR(255),
        link VARCHAR(255),
        date DATE
    );
''')

<psycopg.Cursor [COMMAND_OK] [INTRANS] (host=db.qoleuetswfwqnqbaafpn.supabase.co database=postgres) at 0x2597869bd10>

In [42]:
conn.commit()

In [14]:
cur.close()
conn.close()

In [7]:
response = requests.get("https://u.today/search/node?keys=bitcoin")
document = bs4.BeautifulSoup(response.text)
news_items = document.select("div.search-result > .news__item")

In [8]:
len(news_items)

50

In [9]:
add_to_db = []

for item in news_items:
    try: 
        data = []
        data.append(item.select_one("div.news__item-title").get_text()) # title
        data.append(item.select_one("a.humble.humble--author").get_text().split("\n")[0]) # author
        data.append(item.select_one("div.humble").get_text().split(" - ")[0]) # date
        data.append(item.select_one("a.news__item-body")["href"]) # link
        add_to_db.append(data)
    except: print("value missing")

value missing


In [12]:
for item in add_to_db:
    item[2] = dt.strptime(item[2], "%b %d, %Y")
    cur.execute('''
        INSERT INTO utoday_news (title, author, date, link)
        VALUES (%s, %s, %s, %s);
    ''', item)

print("finished")

finished


In [16]:
cur.execute('''
    ALTER TABLE utoday_news
    ADD sentiment VARCHAR(255);
''')

<psycopg.Cursor [COMMAND_OK] [INTRANS] (host=db.qoleuetswfwqnqbaafpn.supabase.co database=postgres) at 0x25979130050>

In [18]:
titles = cur.execute('''
    SELECT title FROM utoday_news ORDER BY date DESC;
''').fetchall()

titles = [title[0] for title in titles]

In [19]:
hf_api_key = os.getenv("HF_API_KEY")
url = "https://api-inference.huggingface.co/models/mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis"

payload = { "inputs": titles }
headers = { "Authorization": f"Bearer {hf_api_key}" }
response = requests.post(url, headers=headers, json=payload)
print("first response", response)

if response.status_code == 503:
    headers["x-wait-for-model"] = "true"
    response = requests.post(url, headers=headers, json=payload)
    print("second response", response)

first response <Response [200]>


In [32]:
import json

sentiment_results = response.json()[0]
print(json.dumps(sentiment_results, indent=2))

[
  {
    "label": "neutral",
    "score": 0.9846206903457642
  },
  {
    "label": "neutral",
    "score": 0.999874472618103
  },
  {
    "label": "neutral",
    "score": 0.9993852376937866
  },
  {
    "label": "positive",
    "score": 0.9455152153968811
  },
  {
    "label": "positive",
    "score": 0.9992294311523438
  },
  {
    "label": "neutral",
    "score": 0.9998427629470825
  },
  {
    "label": "neutral",
    "score": 0.5809779167175293
  },
  {
    "label": "neutral",
    "score": 0.9993582367897034
  },
  {
    "label": "negative",
    "score": 0.9932060241699219
  },
  {
    "label": "neutral",
    "score": 0.7275463342666626
  },
  {
    "label": "negative",
    "score": 0.9909226298332214
  },
  {
    "label": "neutral",
    "score": 0.9760090112686157
  },
  {
    "label": "neutral",
    "score": 0.9998712539672852
  },
  {
    "label": "neutral",
    "score": 0.999863862991333
  },
  {
    "label": "positive",
    "score": 0.9985255599021912
  },
  {
    "label": "ne

In [33]:
len(sentiment_results), len(titles)

(49, 49)

In [40]:
sentiment_results[0]["label"]

'neutral'

In [ ]:
if len(sentiment_results) == len(titles):
    for i in range(0, len(titles)):
        label = sentiment_results[i]["label"]
        # print(label)
        cur.execute('''
            UPDATE utoday_news
            SET sentiment = %s
            WHERE title = %s;
        ''', (label, titles[i]))

conn.commit()
print("finished")

neutral
neutral
neutral
positive
positive
neutral
neutral
neutral
negative
neutral
negative
neutral
neutral
neutral
positive
neutral
positive
neutral
positive
positive
positive
neutral
neutral
neutral
neutral
negative
negative
neutral
neutral
positive
positive
neutral
neutral
negative
negative
negative
positive
neutral
neutral
neutral
neutral
neutral
positive
neutral
negative
negative
positive
neutral
negative
finished


In [26]:
cur.execute('''
    SELECT * FROM utoday_news ORDER BY date LIMIT 1;
''')
cur.fetchall()

[('$103 Million Bitcoin Transfers Stun Major Crypto Exchange in Minutes',
  'Caroline Amosun',
  'https://u.today/103-million-bitcoin-transfers-stun-major-crypto-exchange-in-minutes',
  datetime.date(2025, 5, 14),
  None)]

In [27]:
cur.execute('''
    SELECT * FROM utoday_news;
''')
cur.fetchall()

[('Bitcoin Enters ATH Season, Samson Mow Believes',
  'Yuri Molchan',
  'https://u.today/bitcoin-enters-ath-season-samson-mow-believes',
  datetime.date(2025, 5, 19),
  None),
 ("'Rich Dad Poor Dad' Author Issues Big Call: Just Bitcoin, No BTC ETFs",
  'Yuri Molchan',
  'https://u.today/rich-dad-poor-dad-author-issues-big-call-just-bitcoin-no-btc-etfs',
  datetime.date(2025, 5, 19),
  None),
 ('Bitcoin Logs All-Time High Weekly Candle',
  'Alex Dovbnya',
  'https://u.today/bitcoin-logs-all-time-high-weekly-candle',
  datetime.date(2025, 5, 19),
  None),
 ("XRP: Pivotal 'Crash or Rally' Moment, Explosive Bitcoin (BTC) Boost Around Corner, Ethereum (ETH): $3,000 Looms?",
  'Arman Shirinyan',
  'https://u.today/xrp-pivotal-crash-or-rally-moment-explosive-bitcoin-btc-boost-around-corner-ethereum-eth-3000-looms',
  datetime.date(2025, 5, 19),
  None),
 ('Bitcoin Steadily Approaches $106,000',
  'Alex Dovbnya',
  'https://u.today/bitcoin-steadily-approaches-106000',
  datetime.date(2025, 5, 

In [ ]:
cur.execute("ROLLBACK")

In [92]:
cur.execute('''
    DELETE FROM utoday_news WHERE author = '';
''')
conn.commit()